In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import pandas as pd
%matplotlib inline

In [ ]:
def read_data():
    
    file_path = f'data_out/backup/S6T33_t_corrected.csv'
    
    usecols = ['y', 'x', 'frame', 'particle']
    
    data = pd.read_csv(file_path, usecols=usecols)
    
    
    return data

In [ ]:
def common_particles(frame1, frame2):
    
    df1 = pd.DataFrame(columns=['particle', 'x', 'y'])
    df2 = pd.DataFrame(columns=['particle', 'x', 'y'])
    
    i = -1
    for part in frame1['particle']:
        
        if (part in frame2['particle'].unique()):
            mask1 = frame1['particle'] == part
            x1 = frame1[mask1]['x'].item()
            y1 = frame1[mask1]['y'].item()

            mask2 = frame2['particle'] == part
            x2 = frame2[mask2]['x'].item()
            y2 = frame2[mask2]['y'].item()

            i += 1
            df1.loc[i] = [int(part), x1, y1]
            df2.loc[i] = [int(part), x2, y2]
    
    
    return df1, df2

In [ ]:
def draw_ball(x, y, radius=0.05, base_color='blue'):
    """
    Draws a ball at the given (x, y) coordinate.
    
    Parameters:
    x (float): The x-coordinate of the ball's center.
    y (float): The y-coordinate of the ball's center.
    radius (float): The radius of the ball.
    color (str): The color of the ball.
    """
    
    # Draw the ball (circle)
    ball = plt.Circle((x, y), radius, color=base_color)
    ax.add_patch(ball)
    
#     # Create a grid of points for the gradient fill
#     theta = np.linspace(0, 2 * np.pi, 100)
#     r = np.linspace(0, radius, 50)
#     T, R = np.meshgrid(theta, r)
    
#     # Convert polar coordinates to Cartesian coordinates
#     X = x + R * np.cos(T)
#     Y = y + R * np.sin(T)
    
#     # Create a shading effect to simulate a light source from the top left
#     Z = np.sqrt(R)  # Using square root gives a gradient effect
    
#     # Create a color map based on the base color
#     base_rgb = mcolors.to_rgb(base_color)  # Convert base color to RGB
#     color_map = plt.cm.ScalarMappable(cmap=plt.cm.Blues)
#     color_map.set_array(Z)
    
#     # Customize the colormap based on the input color
#     custom_cmap = mcolors.LinearSegmentedColormap.from_list("", ["white", base_color])

#     # Plot the filled contour to simulate the sphere
#     contour = ax.contourf(X, Y, Z, levels=50, cmap=custom_cmap)#, 

#     # Add a light spot to simulate the specular reflection (highlight)
#     ax.plot(x + 0.3 * radius * np.cos(theta), y + 0.3 * radius * np.sin(theta), color='white', alpha=0.3, lw=1)

    

#     plt.show()

In [ ]:
%%time


data = read_data()

frame_0 = data[data['frame'] == 0]
frame_1 = data[data['frame'] == 5] #450
frame_2 = data[data['frame'] == 50] #4500
# frame_3 = data[data['frame'] == 4550]

df0, df1 = common_particles(frame_0, frame_1)
_, df2 = common_particles(frame_0, frame_2)
# _, df3 = common_particles(frame_0, frame_3)

# print(df1.head())
# print(df2.head())
print(df0.shape[0])
print(df1.shape[0])
print(df2.shape[0])
# print(df3.shape[0])

In [ ]:
# max_frame = data['frame'].values[-1] + 1
# print(max_frame)

In [ ]:
# Find intersection of all DataFrames on the common column 'particle'

def filter_df(*dfs):
    
    common_part = set(dfs[0]['particle'])
    
    for df in dfs[1:]:
        common_part.intersection_update(set(df['particle']))

    # Filter each DataFrame to keep only rows with particles in the common_parts
    filtered_dfs = [df[df['particle'].isin(common_part)] for df in dfs]
    
    
    return filtered_dfs

In [ ]:
# filtered_dfs = filter_df(df0, df1, df2)

# print(len(filtered_dfs))
# print(len(filtered_dfs[1]))

# print(filtered_dfs[0].head())

In [ ]:
dfs = filter_df(df0, df1, df2)
print(len(dfs[0]))

In [ ]:
def dr(df1, df2):
    
#     a_s = 0.5#1.28 # micron for S6 sample
#     micron_to_px = 1/0.06905
#     a_s_px = a_s * micron_to_px # 18.53
    
    # Merge dataframes on 'particle' to align rows with the same 'particle'
    df = pd.merge(df1, df2, on='particle', suffixes=('1', '2'))
    
    # Calculate the difference in 'x' values for each particle
    df['dr'] = np.sqrt( (df.x2 - df.x1)**2 + (df.y2 - df.y1)**2 )
    
    
#     df = df[df['dr'] >= a_s_px]
    df_sorted = df.sort_values(by='dr')
    N = len(df)
    N_top = math.ceil(N / 10) # choose 10% of the largest displacement
    
    df_top = df_sorted.tail(N_top)
    
    
    return df_top

In [ ]:
df_dr = dr(dfs[0], dfs[2])
# dfs_as = filter_df(df0, df1, df2)
# print(len(dfs[0]))
print(f"N = {len(df_dr)}")
print('-------')
print(f"max = {df_dr['dr'].max()}")
print('-------')
print(df_dr['dr'])

In [ ]:
dfs_top5 = filter_df(dfs[0], dfs[1], dfs[2], df_dr)
print(len(dfs_top5))

In [ ]:
# x_max = 512 # px
# y_max = 128


x0 = dfs_top5[0].x
y0 = dfs_top5[0].y

x1 = dfs_top5[1].x
y1 = dfs_top5[1].y

x2 = dfs_top5[2].x
y2 = dfs_top5[2].y

# x3 = dfs[3].x
# y3 = dfs[3].y

# x0 = x0 / x_max
# y0 = y0 / y_max

# print(x_nd)


fig, ax = plt.subplots(figsize=(15, 10))
ax.set_aspect('equal')

for x_val, y_val in zip(x0, y0):
    draw_ball(x_val, y_val, radius=5, base_color='navy')#('navy', 1))

for x_val, y_val in zip(x1, y1):
    draw_ball(x_val, y_val, radius=5, base_color=('blue', 0.5))#('blue', 0.5))
    
for x_val, y_val in zip(x2, y2):
    draw_ball(x_val, y_val, radius=5, base_color="violet")#('blue', 0.5))
    
# for x_val, y_val in zip(x3, y3):
#     draw_ball(x_val, y_val, radius=5, color="red")#('blue', 0.25))

navy_patch = mpatches.Patch(color='navy', label='t = 0')
blue_patch = mpatches.Patch(color=('blue', 0.5), label=r't =  $t^*$')
violet_patch = mpatches.Patch(color='violet', label=r't =  $10\times t^*$')

plt.legend(handles=[navy_patch, blue_patch, violet_patch], loc='lower center', bbox_to_anchor=(0.5, -0.4))

# plt.xlim(-0.25, 1.25)
# plt.ylim(-0.25, 1.25)
plt.xlim(-10, 520)
plt.ylim(-10, 130)

plt.show()